Spark Sessions

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Read csv file

In [0]:
flight_data=spark.read.format("csv")\
    .option("header","false")\
    .option("inferSchema","false")\
    .option("mode","FAILFAST")\
    .load("/FileStore/tables/2010_summary.csv")

flight_data.show()

+--------------------+-------------------+-----+
|                 _c0|                _c1|  _c2|
+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United State

In [0]:
flight_data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



In [0]:
flight_data_inferschema=spark.read.format("csv")\
    .option("header","false")\
    .option("inferSchema","true")\
    .option("mode","FAILFAST")\
    .load("/FileStore/tables/2010_summary.csv")

flight_data_inferschema.show()

+--------------------+-------------------+-----+
|                 _c0|                _c1|  _c2|
+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United State

In [0]:
flight_data_inferschema.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



# Handling bad Records

In [0]:
#using permissive mode
employee_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferschema", "true") \
    .option("mode", "PERMISSIVE") \
    .load("/FileStore/tables/emp-1.csv")

employee_df.show()

+---+--------+---+------+------------+--------+
| id|    name|age|salary|     address| nominee|
+---+--------+---+------+------------+--------+
|  1|  Manish| 26| 75000|       bihar|nominee1|
|  2|  Nikita| 23|100000|uttarpradesh|nominee2|
|  3|  Pritam| 22|150000|   Bangalore|   India|
|  4|Prantosh| 17|200000|     Kolkata|   India|
|  5|  Vikash| 31|300000|        null|nominee5|
+---+--------+---+------+------------+--------+



In [0]:
#using DROPMALFORMED 
employee_df1 = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferschema", "true") \
    .option("mode", "DROPMALFORMED") \
    .load("/FileStore/tables/emp-1.csv")

employee_df1.show()

+---+------+---+------+------------+--------+
| id|  name|age|salary|     address| nominee|
+---+------+---+------+------------+--------+
|  1|Manish| 26| 75000|       bihar|nominee1|
|  2|Nikita| 23|100000|uttarpradesh|nominee2|
|  5|Vikash| 31|300000|        null|nominee5|
+---+------+---+------+------------+--------+



In [0]:
#using FailFast
employee_df1 = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferschema", "true") \
    .option("mode", "FAILFAST") \
    .load("/FileStore/tables/emp-1.csv")

employee_df1.show()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-4481292838860847>:8
      1 #using FailFast
      2 employee_df1 = spark.read.format("csv") \
      3     .option("header", "true") \
      4     .option("inferschema", "true") \
      5     .option("mode", "FAILFAST") \
      6     .load("/FileStore/tables/emp-1.csv")
----> 8 employee_df1.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeError(
    915         err

- in permisive mode if there are any corupted rows it will print it
- in DROPMALFORMED mode it wont print those rows where corupted data is there
- in FAILFAST mode if any row is corrupted it wont print any thing . it returns error

in above example
- using permisve mode -all rows printed
- using DROPMALFORMED mode - 3 rows printed
- using FailFast - error is returned



# Print Bad Records

In [0]:
from pyspark.sql.types import StringType, StructField, StructType, IntegerType

emp_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("salary", IntegerType(), True),
    StructField("address", StringType(), True),
    StructField("nominee", StringType(), True),
    StructField("_corrupt_record", StringType(),True)
])

In [0]:
employee_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferschema", "true") \
    .option("mode", "PERMISSIVE") \
    .schema(emp_schema) \
    .load("/FileStore/tables/emp-1.csv")

employee_df.show(truncate=False)

+---+--------+---+------+------------+--------+-------------------------------------------+
|id |name    |age|salary|address     |nominee |_corrupt_record                            |
+---+--------+---+------+------------+--------+-------------------------------------------+
|1  |Manish  |26 |75000 |bihar       |nominee1|null                                       |
|2  |Nikita  |23 |100000|uttarpradesh|nominee2|null                                       |
|3  |Pritam  |22 |150000|Bangalore   |India   |3,Pritam,22,150000,Bangalore,India,nominee3|
|4  |Prantosh|17 |200000|Kolkata     |India   |4,Prantosh,17,200000,Kolkata,India,nominee4|
|5  |Vikash  |31 |300000|null        |nominee5|null                                       |
+---+--------+---+------+------------+--------+-------------------------------------------+



store bad records

In [0]:
employee_df_bad_records = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .schema(emp_schema) \
    .option("badRecordsPath", "/FileStore/tables/bad_records") \
    .load("/FileStore/tables/emp-1.csv")

employee_df_bad_records.show(truncate=False)

+---+------+---+------+------------+--------+---------------+
|id |name  |age|salary|address     |nominee |_corrupt_record|
+---+------+---+------+------------+--------+---------------+
|1  |Manish|26 |75000 |bihar       |nominee1|null           |
|2  |Nikita|23 |100000|uttarpradesh|nominee2|null           |
|5  |Vikash|31 |300000|null        |nominee5|null           |
+---+------+---+------+------------+--------+---------------+



In [0]:
%fs
ls /FileStore/tables/bad_records/20240906T082710/bad_records/

path,name,size,modificationTime
dbfs:/FileStore/tables/bad_records/20240906T082710/bad_records/part-00000-eea2385e-bc67-4bde-bf40-7e60e9e571d2,part-00000-eea2385e-bc67-4bde-bf40-7e60e9e571d2,478,1725611232000


In [0]:
bad_data_df=spark.read.format("json").load("/FileStore/tables/bad_records/20240906T082710/bad_records/")
bad_data_df.show(truncate=False)

+--------------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------+
|path                            |reason                                                                                                                          |record                                     |
+--------------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------+
|dbfs:/FileStore/tables/emp-1.csv|org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 3,Pritam,22,150000,Bangalore,India,nominee3|3,Pritam,22,150000,Bangalore,India,nominee3|
|dbfs:/FileStore/tables/emp-1.csv|org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 4,Prantosh,17,200000,Kolkata,India,nominee4|4,Pra

#Json

In [0]:
spark.read.format("json") \
    .option("inferSchema", "true") \
    .option("mode", "PERMISSIVE") \
    .load("/FileStore/tables/line_delimited.json").show()

+---+--------+------+
|age|    name|salary|
+---+--------+------+
| 20|  Manish| 20000|
| 25|  Nikita| 21000|
| 16|  Pritam| 22000|
| 35|Prantosh| 25000|
| 67|  Vikash| 40000|
+---+--------+------+



# Parquet FIle

In [0]:
df=spark.read.parquet("/FileStore/tables/parquet_file_gz.parquet")
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

# How to write dataframe to disk

In [0]:
df=spark.read.format("csv")\
    .option("header","True")\
    .option("inferSchema","false")\
    .option("mode","FAILFAST")\
    .load("/FileStore/tables/write-1.csv")

df.show()

+---+--------+---+------+-------+------+
| id|    name|age|salary|address|gender|
+---+--------+---+------+-------+------+
|  1|  Manish| 26| 75000|  INDIA|     m|
|  2|  Nikita| 23|100000|    USA|     f|
|  3|  Pritam| 22|150000|  INDIA|     m|
|  4|Prantosh| 17|200000|  JAPAN|     m|
|  5|  Vikash| 31|300000|    USA|     m|
|  6|   Rahul| 55|300000|  INDIA|     m|
|  7|    Raju| 67|540000|    USA|     m|
|  8| Praveen| 28| 70000|  JAPAN|     m|
|  9|     Dev| 32|150000|  JAPAN|     m|
| 10|  Sherin| 16| 25000| RUSSIA|     f|
| 11|    Ragu| 12| 35000|  INDIA|     f|
| 12|   Sweta| 43|200000|  INDIA|     f|
| 13| Raushan| 48|650000|    USA|     m|
| 14|  Mukesh| 36| 95000| RUSSIA|     m|
| 15| Prakash| 52|750000|  INDIA|     m|
+---+--------+---+------+-------+------+



In [0]:
df.write.format("csv")\
    .option("header","true")\
    .option("mode","overwrite")\
    .option("path","/FileStore/tables/write")\
    .save()


In [0]:
dbutils.fs.ls("/FileStore/tables/csv_write/")

Out[9]: [FileInfo(path='dbfs:/FileStore/tables/csv_write/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1725625380000),
 FileInfo(path='dbfs:/FileStore/tables/csv_write/_committed_5919529139592831578', name='_committed_5919529139592831578', size=111, modificationTime=1725625379000),
 FileInfo(path='dbfs:/FileStore/tables/csv_write/_started_5919529139592831578', name='_started_5919529139592831578', size=0, modificationTime=1725625378000),
 FileInfo(path='dbfs:/FileStore/tables/csv_write/part-00000-tid-5919529139592831578-4064e869-ffb7-4d7e-a639-d23af5734c67-9-1-c000.csv', name='part-00000-tid-5919529139592831578-4064e869-ffb7-4d7e-a639-d23af5734c67-9-1-c000.csv', size=454, modificationTime=1725625379000)]

# Partition BY

In [0]:
df.write.format("csv")\
    .option("header","true")\
    .option("mode","overwrite")\
    .option("path","/FileStore/tables/partion_by_address/")\
    .partitionBy("address")\
    .save()

In [0]:
dbutils.fs.ls("/FileStore/tables/partion_by_address/")

Out[16]: [FileInfo(path='dbfs:/FileStore/tables/partion_by_address/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1725626740000),
 FileInfo(path='dbfs:/FileStore/tables/partion_by_address/address=INDIA/', name='address=INDIA/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/partion_by_address/address=JAPAN/', name='address=JAPAN/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/partion_by_address/address=RUSSIA/', name='address=RUSSIA/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/partion_by_address/address=USA/', name='address=USA/', size=0, modificationTime=0)]

In [0]:
# under country partioning again partion by gender

df.write.format("csv")\
    .option("header","true")\
    .option("mode","overwrite")\
    .option("path","/FileStore/tables/partion_by_address_gender/")\
    .partitionBy("address","gender")\
    .save()

In [0]:
dbutils.fs.ls("/FileStore/tables/partion_by_address_gender/")

Out[18]: [FileInfo(path='dbfs:/FileStore/tables/partion_by_address_gender/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1725627028000),
 FileInfo(path='dbfs:/FileStore/tables/partion_by_address_gender/address=INDIA/', name='address=INDIA/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/partion_by_address_gender/address=JAPAN/', name='address=JAPAN/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/partion_by_address_gender/address=RUSSIA/', name='address=RUSSIA/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/partion_by_address_gender/address=USA/', name='address=USA/', size=0, modificationTime=0)]

In [0]:
dbutils.fs.ls("dbfs:/FileStore/tables/partion_by_address_gender/address=INDIA/")

Out[20]: [FileInfo(path='dbfs:/FileStore/tables/partion_by_address_gender/address=INDIA/gender=f/', name='gender=f/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/partion_by_address_gender/address=INDIA/gender=m/', name='gender=m/', size=0, modificationTime=0)]

# BucketBy

In [0]:
df.write.format("csv")\
    .option("header","true")\
    .option("mode","overwrite")\
    .option("path","/FileStore/tables/bucketby_id/")\
    .bucketBy(3,"id")\
    .saveAsTable("bucket_by_id_table")

In [0]:
dbutils.fs.ls("/FileStore/tables/bucketby_id/")

Out[24]: [FileInfo(path='dbfs:/FileStore/tables/bucketby_id/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1725627420000),
 FileInfo(path='dbfs:/FileStore/tables/bucketby_id/_committed_3005835375704632374', name='_committed_3005835375704632374', size=306, modificationTime=1725627420000),
 FileInfo(path='dbfs:/FileStore/tables/bucketby_id/_started_3005835375704632374', name='_started_3005835375704632374', size=0, modificationTime=1725627419000),
 FileInfo(path='dbfs:/FileStore/tables/bucketby_id/part-00000-tid-3005835375704632374-9904820d-7863-469c-a295-b444a7a9eb7c-18-1_00000.c000.csv', name='part-00000-tid-3005835375704632374-9904820d-7863-469c-a295-b444a7a9eb7c-18-1_00000.c000.csv', size=239, modificationTime=1725627419000),
 FileInfo(path='dbfs:/FileStore/tables/bucketby_id/part-00000-tid-3005835375704632374-9904820d-7863-469c-a295-b444a7a9eb7c-18-2_00001.c000.csv', name='part-00000-tid-3005835375704632374-9904820d-7863-469c-a295-b444a7a9eb7c-18-2_00001.c000.csv', size=172, mo

# Data Frame API

In [0]:
my_data=[(1,1),
      (2,1),
      (3,1),
      (4,2),
      (5,1),
      (6,2),
      (7,2)]

In [0]:
my_schema= ["id","num"]

In [0]:
spark.createDataFrame(data=my_data,schema=my_schema).show()

+---+---+
| id|num|
+---+---+
|  1|  1|
|  2|  1|
|  3|  1|
|  4|  2|
|  5|  1|
|  6|  2|
|  7|  2|
+---+---+



# DataFrame Transformation Part 1

fetch columnss

In [0]:
employee_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferschema", "true") \
    .option("mode", "PERMISSIVE") \
    .load("/FileStore/tables/emp-1.csv")


employee_df.columns

Out[33]: ['id', 'name', 'age', 'salary', 'address', 'nominee']

create row record/object

In [0]:
from pyspark.sql import Row
row=Row("1","manish","26","44444")

create column objects

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# columns are expressions
# set of transformation on one or more than one value in a record

df.select(col("age")+5)


Multiple ways of selecting columns

In [0]:
employee_df.select("name").show()

+--------+
|    name|
+--------+
|  Manish|
|  Nikita|
|  Pritam|
|Prantosh|
|  Vikash|
+--------+



In [0]:
employee_df.select(col("name")).show()

+--------+
|    name|
+--------+
|  Manish|
|  Nikita|
|  Pritam|
|Prantosh|
|  Vikash|
+--------+



In [0]:
employee_df.select(col("id")+5).show()

+--------+
|(id + 5)|
+--------+
|       6|
|       7|
|       8|
|       9|
|      10|
+--------+



In [0]:
employee_df.select("id","name","age").show()

+---+--------+---+
| id|    name|age|
+---+--------+---+
|  1|  Manish| 26|
|  2|  Nikita| 23|
|  3|  Pritam| 22|
|  4|Prantosh| 17|
|  5|  Vikash| 31|
+---+--------+---+



In [0]:
employee_df.select(employee_df["salary"]).show()

+------+
|salary|
+------+
| 75000|
|100000|
|150000|
|200000|
|300000|
+------+



In [0]:
employee_df.select(employee_df.address).show()

+------------+
|     address|
+------------+
|       bihar|
|uttarpradesh|
|   Bangalore|
|     Kolkata|
|        null|
+------------+



In [0]:
employee_df.select(expr("id +5")).show()

+--------+
|(id + 5)|
+--------+
|       6|
|       7|
|       8|
|       9|
|      10|
+--------+



# spark sql

In [0]:
# create a view
employee_df.createOrReplaceTempView("employee_tbl")


In [0]:
spark.sql(""" select * from employee_tbl""").show()

+---+--------+---+------+------------+--------+
| id|    name|age|salary|     address| nominee|
+---+--------+---+------+------------+--------+
|  1|  Manish| 26| 75000|       bihar|nominee1|
|  2|  Nikita| 23|100000|uttarpradesh|nominee2|
|  3|  Pritam| 22|150000|   Bangalore|   India|
|  4|Prantosh| 17|200000|     Kolkata|   India|
|  5|  Vikash| 31|300000|        null|nominee5|
+---+--------+---+------+------------+--------+



# Data frame Transpormation part 2

In [0]:

# ALiase
employee_df.select(col("id").alias("employee_id"),"name","age").show()

+-----------+--------+---+
|employee_id|    name|age|
+-----------+--------+---+
|          1|  Manish| 26|
|          2|  Nikita| 23|
|          3|  Pritam| 22|
|          4|Prantosh| 17|
|          5|  Vikash| 31|
+-----------+--------+---+



filter

In [0]:
employee_df.filter((col("salary")>150000) & (col("age")<18)).show()

+---+--------+---+------+-------+-------+
| id|    name|age|salary|address|nominee|
+---+--------+---+------+-------+-------+
|  4|Prantosh| 17|200000|Kolkata|  India|
+---+--------+---+------+-------+-------+



In [0]:
# add columns
employee_df.withColumn("sir_name",lit("singh")).show()

+---+--------+---+------+------------+--------+--------+
| id|    name|age|salary|     address| nominee|sir_name|
+---+--------+---+------+------------+--------+--------+
|  1|  Manish| 26| 75000|       bihar|nominee1|   singh|
|  2|  Nikita| 23|100000|uttarpradesh|nominee2|   singh|
|  3|  Pritam| 22|150000|   Bangalore|   India|   singh|
|  4|Prantosh| 17|200000|     Kolkata|   India|   singh|
|  5|  Vikash| 31|300000|        null|nominee5|   singh|
+---+--------+---+------+------------+--------+--------+



In [0]:
#rename column
employee_df.withColumnRenamed("id","employer_id").show()

+-----------+--------+---+------+------------+--------+
|employer_id|    name|age|salary|     address| nominee|
+-----------+--------+---+------+------------+--------+
|          1|  Manish| 26| 75000|       bihar|nominee1|
|          2|  Nikita| 23|100000|uttarpradesh|nominee2|
|          3|  Pritam| 22|150000|   Bangalore|   India|
|          4|Prantosh| 17|200000|     Kolkata|   India|
|          5|  Vikash| 31|300000|        null|nominee5|
+-----------+--------+---+------+------------+--------+



In [0]:
employee_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- address: string (nullable = true)
 |-- nominee: string (nullable = true)



In [0]:
# type cast
employee_df.withColumn("id",col("id").cast("string"))\
    .withColumn("salary",col("salary").cast("long"))\
.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: long (nullable = true)
 |-- address: string (nullable = true)
 |-- nominee: string (nullable = true)



In [0]:
# remove column
employee_df.drop("id",col("name")).show()

+---+------+------------+--------+
|age|salary|     address| nominee|
+---+------+------------+--------+
| 26| 75000|       bihar|nominee1|
| 23|100000|uttarpradesh|nominee2|
| 22|150000|   Bangalore|   India|
| 17|200000|     Kolkata|   India|
| 31|300000|        null|nominee5|
+---+------+------------+--------+



# Union ANd UNion All

In [0]:

data=[(10 ,'Anil',50000, 18),
(11 ,'Vikas',75000,  16),
(12 ,'Nisha',40000,  18),
(13 ,'Nidhi',60000,  17),
(14 ,'Priya',80000,  18),
(15 ,'Mohit',45000,  18),
(16 ,'Rajesh',90000, 10),
(17 ,'Raman',55000, 16),
(18 ,'Sam',65000,   17),
(18 ,'Sam',65000,   17),
(18 ,'Sam',65000,   17)]

schema=['id','Name','sal','mngr_id']

manager_df=spark.createDataFrame(data,schema=schema)


In [0]:
manager_df.show()

+---+------+-----+-------+
| id|  Name|  sal|mngr_id|
+---+------+-----+-------+
| 10|  Anil|50000|     18|
| 11| Vikas|75000|     16|
| 12| Nisha|40000|     18|
| 13| Nidhi|60000|     17|
| 14| Priya|80000|     18|
| 15| Mohit|45000|     18|
| 16|Rajesh|90000|     10|
| 17| Raman|55000|     16|
| 18|   Sam|65000|     17|
| 18|   Sam|65000|     17|
| 18|   Sam|65000|     17|
+---+------+-----+-------+



In [0]:
manager_df.count()

Out[26]: 9

In [0]:
data1=[(19 ,'Sohan',50000, 18),
(20 ,'Sima',75000,  17)]

schema1=['id','Name','sal','mngr_id']

manager_df1=spark.createDataFrame(data=data1,schema=schema)

manager_df1.show()

+---+-----+-----+-------+
| id| Name|  sal|mngr_id|
+---+-----+-----+-------+
| 19|Sohan|50000|     18|
| 20| Sima|75000|     17|
+---+-----+-----+-------+



In [0]:
#union
manager_df.union(manager_df1).show()

+---+------+-----+-------+
| id|  Name|  sal|mngr_id|
+---+------+-----+-------+
| 10|  Anil|50000|     18|
| 11| Vikas|75000|     16|
| 12| Nisha|40000|     18|
| 13| Nidhi|60000|     17|
| 14| Priya|80000|     18|
| 15| Mohit|45000|     18|
| 16|Rajesh|90000|     10|
| 17| Raman|55000|     16|
| 18|   Sam|65000|     17|
| 19| Sohan|50000|     18|
| 20|  Sima|75000|     17|
+---+------+-----+-------+



In [0]:
#union all
manager_df.unionAll(manager_df1).show()

+---+------+-----+-------+
| id|  Name|  sal|mngr_id|
+---+------+-----+-------+
| 10|  Anil|50000|     18|
| 11| Vikas|75000|     16|
| 12| Nisha|40000|     18|
| 13| Nidhi|60000|     17|
| 14| Priya|80000|     18|
| 15| Mohit|45000|     18|
| 16|Rajesh|90000|     10|
| 17| Raman|55000|     16|
| 18|   Sam|65000|     17|
| 18|   Sam|65000|     17|
| 18|   Sam|65000|     17|
| 19| Sohan|50000|     18|
| 20|  Sima|75000|     17|
+---+------+-----+-------+



# case

In [0]:
emp_data = [
(1,'manish',26,20000,'india','IT'),
(2,'rahul',None,40000,'germany','engineering'),
(3,'pawan',12,60000,'india','sales'),
(4,'roshini',44,None,'uk','engineering'),
(5,'raushan',35,70000,'india','sales'),
(6,None,29,200000,'uk','IT'),
(7,'adam',37,65000,'us','IT'),
(8,'chris',16,40000,'us','sales'),
(None,None,None,None,None,None),
(7,'adam',37,65000,'us','IT')
]

schema1=['id','Name','age','salary','country','dept']
emp_df=spark.createDataFrame(data=emp_data,schema=schema1)
emp_df.show()

+----+-------+----+------+-------+-----------+
|  id|   Name| age|salary|country|       dept|
+----+-------+----+------+-------+-----------+
|   1| manish|  26| 20000|  india|         IT|
|   2|  rahul|null| 40000|germany|engineering|
|   3|  pawan|  12| 60000|  india|      sales|
|   4|roshini|  44|  null|     uk|engineering|
|   5|raushan|  35| 70000|  india|      sales|
|   6|   null|  29|200000|     uk|         IT|
|   7|   adam|  37| 65000|     us|         IT|
|   8|  chris|  16| 40000|     us|      sales|
|null|   null|null|  null|   null|       null|
|   7|   adam|  37| 65000|     us|         IT|
+----+-------+----+------+-------+-----------+



In [0]:
emp_df.withColumn("adult",when(col("age")<18,"No")
    .when(col("age")>18,"Yes")
    .otherwise("Novalue")).show()

+----+-------+----+------+-------+-----------+-------+
|  id|   Name| age|salary|country|       dept|  adult|
+----+-------+----+------+-------+-----------+-------+
|   1| manish|  26| 20000|  india|         IT|    Yes|
|   2|  rahul|null| 40000|germany|engineering|Novalue|
|   3|  pawan|  12| 60000|  india|      sales|     No|
|   4|roshini|  44|  null|     uk|engineering|    Yes|
|   5|raushan|  35| 70000|  india|      sales|    Yes|
|   6|   null|  29|200000|     uk|         IT|    Yes|
|   7|   adam|  37| 65000|     us|         IT|    Yes|
|   8|  chris|  16| 40000|     us|      sales|     No|
|null|   null|null|  null|   null|       null|Novalue|
|   7|   adam|  37| 65000|     us|         IT|    Yes|
+----+-------+----+------+-------+-----------+-------+



In [0]:
emp_df.withColumn(
    "age_wise",
    when((col("age") > 0) & (col("age") < 18), "Minor")
    .when((col("age") >= 18) & (col("age") < 30), "Mid")
    .otherwise("Major")
).show()

+----+-------+----+------+-------+-----------+--------+
|  id|   Name| age|salary|country|       dept|age_wise|
+----+-------+----+------+-------+-----------+--------+
|   1| manish|  26| 20000|  india|         IT|     Mid|
|   2|  rahul|null| 40000|germany|engineering|   Major|
|   3|  pawan|  12| 60000|  india|      sales|   Minor|
|   4|roshini|  44|  null|     uk|engineering|   Major|
|   5|raushan|  35| 70000|  india|      sales|   Major|
|   6|   null|  29|200000|     uk|         IT|     Mid|
|   7|   adam|  37| 65000|     us|         IT|   Major|
|   8|  chris|  16| 40000|     us|      sales|   Minor|
|null|   null|null|  null|   null|       null|   Major|
|   7|   adam|  37| 65000|     us|         IT|   Major|
+----+-------+----+------+-------+-----------+--------+



# Unique and sorted

In [0]:
data=[(10 ,'Anil',50000, 18),
(11 ,'Vikas',75000,  16),
(12 ,'Nisha',40000,  18),
(13 ,'Nidhi',60000,  17),
(14 ,'Priya',80000,  18),
(15 ,'Mohit',45000,  18),
(16 ,'Rajesh',90000, 10),
(17 ,'Raman',55000, 16),
(18 ,'Sam',65000,   17),
(15 ,'Mohit',45000,  18),
(13 ,'Nidhi',60000,  17),      
(14 ,'Priya',90000,  18),  
(18 ,'Sam',65000,   17)]

schema=['id','Name','sal','mngr_id']
manager_df=spark.createDataFrame(data=data,schema=schema)
manager_df.show()

+---+------+-----+-------+
| id|  Name|  sal|mngr_id|
+---+------+-----+-------+
| 10|  Anil|50000|     18|
| 11| Vikas|75000|     16|
| 12| Nisha|40000|     18|
| 13| Nidhi|60000|     17|
| 14| Priya|80000|     18|
| 15| Mohit|45000|     18|
| 16|Rajesh|90000|     10|
| 17| Raman|55000|     16|
| 18|   Sam|65000|     17|
| 15| Mohit|45000|     18|
| 13| Nidhi|60000|     17|
| 14| Priya|90000|     18|
| 18|   Sam|65000|     17|
+---+------+-----+-------+



In [0]:

# unique records
manager_df.distinct().show()

+---+------+-----+-------+
| id|  Name|  sal|mngr_id|
+---+------+-----+-------+
| 10|  Anil|50000|     18|
| 12| Nisha|40000|     18|
| 11| Vikas|75000|     16|
| 13| Nidhi|60000|     17|
| 15| Mohit|45000|     18|
| 14| Priya|80000|     18|
| 16|Rajesh|90000|     10|
| 17| Raman|55000|     16|
| 18|   Sam|65000|     17|
| 14| Priya|90000|     18|
+---+------+-----+-------+



In [0]:
manager_df.distinct().count()

Out[7]: 10

In [0]:
manager_df.select("id","name").distinct().show()

+---+------+
| id|  name|
+---+------+
| 10|  Anil|
| 11| Vikas|
| 12| Nisha|
| 13| Nidhi|
| 15| Mohit|
| 14| Priya|
| 17| Raman|
| 16|Rajesh|
| 18|   Sam|
+---+------+



In [0]:
# remove duplicate data
dropped_mngr_data = manager_df.drop_duplicates(["id","name","sal","mngr_id"]).show()

+---+------+-----+-------+
| id|  Name|  sal|mngr_id|
+---+------+-----+-------+
| 10|  Anil|50000|     18|
| 12| Nisha|40000|     18|
| 11| Vikas|75000|     16|
| 13| Nidhi|60000|     17|
| 15| Mohit|45000|     18|
| 14| Priya|80000|     18|
| 16|Rajesh|90000|     10|
| 17| Raman|55000|     16|
| 18|   Sam|65000|     17|
| 14| Priya|90000|     18|
+---+------+-----+-------+



In [0]:
# sort
manager_df.sort(col("sal").desc()).show()

+---+------+-----+-------+
| id|  Name|  sal|mngr_id|
+---+------+-----+-------+
| 16|Rajesh|90000|     10|
| 14| Priya|90000|     18|
| 14| Priya|80000|     18|
| 11| Vikas|75000|     16|
| 18|   Sam|65000|     17|
| 18|   Sam|65000|     17|
| 13| Nidhi|60000|     17|
| 13| Nidhi|60000|     17|
| 17| Raman|55000|     16|
| 10|  Anil|50000|     18|
| 15| Mohit|45000|     18|
| 15| Mohit|45000|     18|
| 12| Nisha|40000|     18|
+---+------+-----+-------+



In [0]:
# multi column sort
manager_df.sort(col("sal").desc(),col("name").desc()).show()

+---+------+-----+-------+
| id|  Name|  sal|mngr_id|
+---+------+-----+-------+
| 16|Rajesh|90000|     10|
| 14| Priya|90000|     18|
| 14| Priya|80000|     18|
| 11| Vikas|75000|     16|
| 18|   Sam|65000|     17|
| 18|   Sam|65000|     17|
| 13| Nidhi|60000|     17|
| 13| Nidhi|60000|     17|
| 17| Raman|55000|     16|
| 10|  Anil|50000|     18|
| 15| Mohit|45000|     18|
| 15| Mohit|45000|     18|
| 12| Nisha|40000|     18|
+---+------+-----+-------+



# Aggreate Functions

In [0]:
emp_data = [
(1,'manish',26,20000,'india','IT'),
(2,'rahul',None,40000,'germany','engineering'),
(3,'pawan',12,60000,'india','sales'),
(4,'roshini',44,None,'uk','engineering'),
(5,'raushan',35,70000,'india','sales'),
(6,None,29,200000,'uk','IT'),
(7,'adam',37,65000,'us','IT'),
(8,'chris',16,40000,'us','sales'),
(None,None,None,None,None,None),
(7,'adam',37,65000,'us','IT')
]

emp_schema=['id','name','age','salary','country','dept']

emp_df=spark.createDataFrame(data=emp_data,schema=emp_schema)

emp_df.show()

+----+-------+----+------+-------+-----------+
|  id|   name| age|salary|country|       dept|
+----+-------+----+------+-------+-----------+
|   1| manish|  26| 20000|  india|         IT|
|   2|  rahul|null| 40000|germany|engineering|
|   3|  pawan|  12| 60000|  india|      sales|
|   4|roshini|  44|  null|     uk|engineering|
|   5|raushan|  35| 70000|  india|      sales|
|   6|   null|  29|200000|     uk|         IT|
|   7|   adam|  37| 65000|     us|         IT|
|   8|  chris|  16| 40000|     us|      sales|
|null|   null|null|  null|   null|       null|
|   7|   adam|  37| 65000|     us|         IT|
+----+-------+----+------+-------+-----------+



In [0]:
emp_df.count()

Out[16]: 10

In [0]:
# indiviudal select

emp_df.select(count("name")).show()

+-----------+
|count(name)|
+-----------+
|          8|
+-----------+



In [0]:
#min , ax , sum

emp_df.select(sum("salary"),max("salary"),min("salary")).show()

+-----------+-----------+-----------+
|sum(salary)|max(salary)|min(salary)|
+-----------+-----------+-----------+
|     560000|     200000|      20000|
+-----------+-----------+-----------+



# Group BY

In [0]:
emp_data = [
    (1, 'manish', 50000, 'IT'),
    (2, 'vikash', 60000, 'sales'),
    (3, 'raushan', 70000, 'marketing'),
    (4, 'mukesh', 80000, 'IT'),
    (5, 'pritam', 90000, 'sales'),
    (6, 'nikita', 45000, 'marketing'),
    (7, 'ragini', 55000, 'marketing'),
    (8, 'rakesh', 100000, 'IT'),
    (9, 'aditya', 65000, 'IT'),
    (10, 'rahul', 50000, 'marketing')
]

emp_schema=['id','name','salary','dept']

emp_df = spark.createDataFrame(data=emp_data,schema=emp_schema)

emp_df.show()



+---+-------+------+---------+
| id|   name|salary|     dept|
+---+-------+------+---------+
|  1| manish| 50000|       IT|
|  2| vikash| 60000|    sales|
|  3|raushan| 70000|marketing|
|  4| mukesh| 80000|       IT|
|  5| pritam| 90000|    sales|
|  6| nikita| 45000|marketing|
|  7| ragini| 55000|marketing|
|  8| rakesh|100000|       IT|
|  9| aditya| 65000|       IT|
| 10|  rahul| 50000|marketing|
+---+-------+------+---------+



In [0]:
emp_df.groupby("dept")\
    .agg(sum("salary")).show()

+---------+-----------+
|     dept|sum(salary)|
+---------+-----------+
|       IT|     295000|
|    sales|     150000|
|marketing|     220000|
+---------+-----------+



In [0]:
spark.sql("""
          select dept,sum("salary")
          from table 
          group by dept 
          """ )

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3854128031212148>:1
----> 1 spark.sql("""
      2           select dept,sum("salary")
      3           from table 
      4           group by dept 
      5           group""" )

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1387, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1385 try:
   1386     litArgs = {k: _to_java_column(lit(v)) for k, v in (args or {}).items()}
-> 1387     return DataFrame(self._jsparkSession.sql(sqlQuery, litArgs), s

# Joins

In [0]:
sales_data = [(1,22,10,"01-06-2022"),
(1,27,5,"03-02-2023"),
(2,5,3,"01-06-2023"),
(5,22,1,"22-03-2023"),
(7,22,4,"03-02-2023"),
(9,5,6,"03-03-2023"),
(2,1,12,"15-06-2023"),
(1,56,2,"25-06-2023"),
(5,12,5,"15-04-2023"),
(11,12,76,"12-03-2023")]

sales_schema=['customer_id','product_id','quantity','date_of_purchase']

sales_df=spark.createDataFrame(data=emp_data,schema=sales_schema)

sales_df.show()

+-----------+----------+--------+----------------+
|customer_id|product_id|quantity|date_of_purchase|
+-----------+----------+--------+----------------+
|          1|    manish|   50000|              IT|
|          2|    vikash|   60000|           sales|
|          3|   raushan|   70000|       marketing|
|          4|    mukesh|   80000|              IT|
|          5|    pritam|   90000|           sales|
|          6|    nikita|   45000|       marketing|
|          7|    ragini|   55000|       marketing|
|          8|    rakesh|  100000|              IT|
|          9|    aditya|   65000|              IT|
|         10|     rahul|   50000|       marketing|
+-----------+----------+--------+----------------+

